In [ ]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()
import sagemaker
import os

sess    = sagemaker.Session()
role    = os.environ.get('SAGEMAKER_EXCECUTION_ROLE')
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region  = sess.boto_session.region_name

### Common settings

In [ ]:
repo_name = 'keras-tf-cpu' # ECR repository
image_tag = 'keras-tf-cpu-py3' # ECR image tag
base_job_name = 'keras-demo-job' # SageMaker training prefix

train_instance_type = 'ml.c5.xlarge'
cpu_count           = 1
batch_size          = 128*cpu_count

In [ ]:
#Download data into work_dir/input/data/training
%env account {account}
%env region {region}
%env repo_name {repo_name}
%env image_tag {image_tag}

### Download Data

In [ ]:
!mkdir -p work_dir/input/data/training

In [ ]:
%%capture
!cd ../work_dir/input/data/training && kaggle datasets download vbookshelf/v2-plant-seedlings-dataset
!unzip ../work_dir/input/data/training/v2-plant-seedlings-dataset.zip -d ../work_dir/input/data/training
!rm ../work_dir/input/data/training/v2-plant-seedlings-dataset

### Upload Data to S3

In [ ]:
import sys,os

local_directory = os.path.realpath("../work_dir/input/data")
prefix          = repo_name+'/input'

train_input_path      = sess.upload_data(local_directory+'/training/',      key_prefix=prefix+'/train')

### Create repository and login

In [ ]:
%%sh

aws ecr describe-repositories --repository-names $repo_name > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name $repo_name > /dev/null
fi

$(aws ecr get-login --region $region --no-include-email)

## Build and tag Docker image

In [ ]:
%%capture
%cd build
!docker build -t $image_tag .
%cd ..    

In [ ]:
!docker tag $image_tag ${account}.dkr.ecr.us-east-1.amazonaws.com/keras-tf-cpu:latest

## Push Docker image to ECR

In [ ]:
!docker push ${account}.dkr.ecr.us-east-1.amazonaws.com/keras-tf-cpu:latest

In [ ]:
output_path = 's3://{}/{}/output'.format(sess.default_bucket(), repo_name)
image_name  = '{}.dkr.ecr.{}.amazonaws.com/{}:cpu'.format(account, region, repo_name)

print(output_path)
print(image_name)

estimator = sagemaker.estimator.Estimator(
                       image_name,
                       base_job_name=base_job_name,
                       role=role, 
                       train_instance_count=1, 
                       train_instance_type=train_instance_type,
                       output_path=output_path,
                       sagemaker_session=sess)

In [ ]:
estimator.fit({'training': train_input_path})

In [ ]:
endpoint_name = 'keras-poc'

predictor = estimator.deploy(initial_instance_count=1, 
             instance_type='ml.m4.xlarge', 
             endpoint_name=endpoint_name)

In [ ]:
sagemaker.delete_endpoint(endpoint_name=endpoint_name)